# Vacancy and Substitutional defect pair in GaN

Focusing on recreating the defect pair in GaN from the Figure 1. c) in the publication:

<img src="https://i.imgur.com/1JsDwXO.png" width="400" />

## 1. Prepare the Environment
### 1.1. Set up defects parameters 
Defect Configuration parameters are described in [Defect Configuration](https://github.com/Exabyte-io/made/blob/8196b759242551c77d1791bf5bd2f4150763cfef/src/py/mat3ra/made/tools/build/defect/configuration.py#L102).

In [ ]:
SUPERCELL_MATRIX = [[3, 0, 0], [0, 3, 0], [0, 0, 2]]

# List of dictionaries with defect parameters
PRIMARY_DEFECT_CONFIG = {
    "defect_type": "substitution",
    "approximate_coordinate": [0.5, 0.5, 0.5],
    "chemical_element": "Mg",
}

SECONDARY_DEFECT_CONFIG = {
    "defect_type": "vacancy",
    "approximate_coordinate": [0.5, 0.5, 0.65],
}

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("specific_examples|create_point_defect.ipynb")

### 1.3. Get input materials
Materials are loaded with `get_materials()`.

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

material = Material(Materials.get_by_name_first_match("GaN"))

### 1.4. Create and preview Supercell

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.supercell import create_supercell

unit_cell = material
supercell = create_supercell(unit_cell, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create the Defect
### 2.1. Initialize Configuration and Builder parameters

In [ ]:
from mat3ra.made.tools.build.defect.configuration import PointDefectPairConfiguration, PointDefectConfiguration
from mat3ra.made.tools.build.defect.builders import PointDefectBuilderParameters

primary_defect_configuration = PointDefectConfiguration.from_dict(supercell, PRIMARY_DEFECT_CONFIG)
secondary_defect_configuration = PointDefectConfiguration.from_dict(supercell, SECONDARY_DEFECT_CONFIG)

point_defect_pair_configuration = PointDefectPairConfiguration(
    primary_defect_configuration=primary_defect_configuration,
    secondary_defect_configuration=secondary_defect_configuration
)

defect_builder_parameters = PointDefectBuilderParameters(center_defect=False)

### 2.2. Create the defects

In [ ]:
from mat3ra.made.tools.build.defect.builders import PointDefectPairBuilder

material_with_defect = PointDefectPairBuilder(defect_builder_parameters).get_material(point_defect_pair_configuration)

## 3. Visualize Result(s)

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}],
          rotation="-90x")
visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import download_content_to_file

download_content_to_file(material_with_defect, "Mg substitution and vacancy in GaN.json")